## 0 - Imports 


In [25]:
# Hugging Face
from datasets import load_dataset
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration

# Data Analysis
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score


# Audio handling
import torch
import soundfile as sf
import librosa

# Audio Recording
import sounddevice as sd
from scipy.io.wavfile import write

# Transcription
from jiwer import wer
from sklearn.feature_extraction.text import TfidfVectorizer


# Misc
import time
import os
import joblib


## 1 - Load Datasets

In [2]:
#Load the training portion of each language - takes a couple minutes
fleurs_en = load_dataset("google/fleurs", "en_us", split="train",trust_remote_code = True)
fleurs_es = load_dataset("google/fleurs", "es_419", split="train", trust_remote_code = True)
fleurs_ja = load_dataset("google/fleurs", "ja_jp", split="train", trust_remote_code = True)


In [3]:
# See outputs
sample_en = fleurs_en[0]
sample_en.keys()

sample_en

#key_features = ['audio' , 'transcription', 'language']

{'id': 903,
 'num_samples': 108800,
 'path': 'C:\\Users\\jking36\\.cache\\huggingface\\datasets\\downloads\\extracted\\00f4c8069bdbe2061746b64297f5f7dd1af5f4ac9c3f84e1cc27006199d80190\\10004088536354799741.wav',
 'audio': {'path': 'train/10004088536354799741.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -3.15904617e-06, -3.03983688e-06, -3.27825546e-06], shape=(108800,)),
  'sampling_rate': 16000},
 'transcription': 'a tornado is a spinning column of very low-pressure air which sucks the surrounding air inward and upward',
 'raw_transcription': 'A tornado is a spinning column of very low-pressure air, which sucks the surrounding air inward and upward.',
 'gender': 1,
 'lang_id': 19,
 'language': 'English',
 'lang_group_id': 0}

## 2 - Obtain ASR Model


In [4]:
device = 0 if torch.cuda.is_available() else -1 

asr = pipeline(
    'automatic-speech-recognition',
    model = 'openai/whisper-small',
    device = device
)

# Or Directly import model

model_name = 'openai/whisper-small'

processor = WhisperProcessor.from_pretrained(model_name)
whisper = WhisperForConditionalGeneration.from_pretrained(model_name)
whisper.eval()



Device set to use cpu


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [5]:
#Use English sample to test asr

audio = sample_en['audio']['array']

testing = asr(audio)

print(f'Predicted Text: {testing["text"]} - Predicted Language: {testing.get("language")}')
print(f'True ENG Text: {sample_en["transcription"]} - True Language: {sample_en["language"]}')

`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


Predicted Text:  A tornado is a spinning column of very low pressure air, which sucks the surrounding air inward and upward. - Predicted Language: None
True ENG Text: a tornado is a spinning column of very low-pressure air which sucks the surrounding air inward and upward - True Language: English


In [6]:
#Function for Language Detection
def detect_language(dataset,n=50):
    true = 0
    for i in range(n):
        sample = dataset[i]
        prediction = asr(sample['audio']['array'])
        if prediction.get('language') == sample['language']:
            true += 1
        print(prediction.get('language'))
        print(sample['language'])
    return true/n

In [7]:
print('English Language Detection Accuracy: ', detect_language(fleurs_en,1))

None
English
English Language Detection Accuracy:  0.0


## 3 - Record Audio for Model
    Function to record audio clip for Language Detection and Transcription

In [55]:
#### RECORD AUDIO FILE .wav
SAMPLE_RATE = 16000
DURATION = 30 #seconds
OUTPUT = 'recording.wav'

print('Recording in process...')

audio = sd.rec(
    int(DURATION * SAMPLE_RATE),
    samplerate = SAMPLE_RATE,
    channels = 1,
    dtype = np.float32
)

sd.wait()

write(OUTPUT,SAMPLE_RATE,audio)
print('Audio Recording Saved --> "recording.wav"')


Recording in process...
Audio Recording Saved --> "recording.wav"


In [56]:
# Transcribe Audio Recording
# --> RUNTIME = 13 sec
recording, sr = librosa.load('recording.wav',sr = 16000)
transcribed = asr(recording)
ttext = transcribed['text']
print(f"Transcription of Recording audio: {ttext}")



Transcription of Recording audio: 英語できますか?英語できます。ありがとうございます。とてかい。つゆき。こんにちは。おにがします。


## 4 - Language Detection Model 
- Basic Classification Model trained on fleurs data
- Goal: Take in transcription of audio recording and accurately display the language
- For: The toggle between languages once we get to the translation model

In [ ]:
#Create Training Data - combination of language data

def fleurs_df(dataset):
    return pd.DataFrame({'text': dataset['transcription'],
                         'language': dataset['language']})

df_en = fleurs_df(fleurs_en)
df_es = fleurs_df(fleurs_es)
df_jp = fleurs_df(fleurs_ja)

data = pd.concat([df_en,df_es,df_jp],ignore_index=True)

data

# data.loc[data['language']=='Japanese']


,text,language
0,a tornado is a spinning column of very low-pre...,English
1,former u.s. speaker of the house newt gingrich...,English
2,the island was first inhabited by the taínos a...,English
3,these nerve impulses can be sent so quickly th...,English
4,on september 24 1759 arthur guinness signed a ...,English
...,...,...
7685,映画をdvd形式に変換しようとするときに最も一般的な問題の1つはオーバースキャンです,Japanese
7686,"地球上で非常に希少な岩石の一部は オンス当たり11,000～22,500米ドルで取引されてお...",Japanese
7687,一部の研究は 世界的な流行になる前に この病気による致死性を低下させなければならないと示唆し...,Japanese
7688,寺院の一部として ジグラットと呼ばれるピラミッド型の特殊な塔が建てられることがありました,Japanese


In [24]:
#Train/Test Split

X = data['text']
y= data['language']

X_train, X_dev, y_train, y_dev = train_test_split(X,y,test_size=.2,random_state = 42, stratify = y)


In [ ]:
# Vectorizer - maybe go back and save on joblib**

vectorized = TfidfVectorizer(
    analyzer='char',
    ngram_range=(2,5),
    min_df=2,
    max_features=20000
)

X_train_v = vectorized.fit_transform(X_train)
X_dev_v = vectorized.transform(X_dev)

X_train_v


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1628999 stored elements and shape (6152, 20000)>

In [42]:
# Random Forest Classifier
def Detection(X_train, X_dev, y_train, y_dev, model_path = 'detection.pkl'):
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

    model.fit(X_train,y_train)
    #Predictions
    y_pred = model.predict(X_dev)
    prob = model.predict_proba(X_dev)[:,1]

    #Scoring - F1/accuracy
    f1 = f1_score(y_dev,y_pred, average= 'macro')
    accuracy = accuracy_score(y_dev,y_pred)
    print(f'F1 Score: {f1:.4f} ---- Accuracy: {accuracy:.4f}')
    print(f'Classification Report: \n {classification_report(y_dev,y_pred)}')

    #Save Model
    joblib.dump(model, model_path)
    print(f"Model saved to {model_path}")

    return model


In [43]:
# Run Training Model 

Detection(X_train_v,X_dev_v, y_train, y_dev)


F1 Score: 0.9993 ---- Accuracy: 0.9993
Classification Report: 
               precision    recall  f1-score   support

     English       1.00      1.00      1.00       521
    Japanese       1.00      1.00      1.00       458
     Spanish       1.00      1.00      1.00       559

    accuracy                           1.00      1538
   macro avg       1.00      1.00      1.00      1538
weighted avg       1.00      1.00      1.00      1538

Model saved to detection.pkl


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [61]:
# Predict Function on Audio Transcription

def predict_detect(text, model_path = 'detection.pkl'):
    model = joblib.load(model_path)
    input = vectorized.transform([text])

    result = model.predict(input)[0]
    pred = model.predict_proba(input)[0]
    conf = dict(zip(model.classes_,pred))
    return result, conf



In [65]:
# Using predict Method on Audio Transcription

language, prob = predict_detect('hola como estas por favor') #basic str example
print(f'Basic String Example: {language}')
print('\n',prob)


language, prob = predict_detect(ttext)
print(f'\n Audio Recording Transcription Detected Language: {language}')
print('\n', prob)




Basic String Example: Spanish

 {'English': np.float64(0.19), 'Japanese': np.float64(0.18), 'Spanish': np.float64(0.63)}

 Audio Recording Transcription Detected Language: Japanese

 {'English': np.float64(0.0), 'Japanese': np.float64(1.0), 'Spanish': np.float64(0.0)}
